# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = '../WeatherPy/weather_output.csv'
weather_df = pd.read_csv(path, encoding = 'utf-8')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,deming,32.2687,-107.7586,87.80,8,1,1621122586,18.41,US
1,ushuaia,-54.8000,-68.3000,42.80,70,40,1621122294,8.05,AR
2,port hardy,50.6996,-127.4199,57.20,72,75,1621122586,12.66,CA
3,torbay,47.6666,-52.7314,41.00,100,75,1621122586,3.44,CA
4,bluff,-46.6000,168.3333,48.00,82,37,1621122587,5.01,NZ
...,...,...,...,...,...,...,...,...,...
544,gao,16.6362,1.6370,88.88,27,40,1621123266,2.33,ML
545,abeche,13.8292,20.8324,81.28,7,1,1621123267,8.19,TD
546,ntchisi,-13.5278,33.9149,53.85,78,47,1621123267,1.36,MW
547,korla,41.7597,86.1469,61.77,39,100,1621123267,11.30,CN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#fig = gmap.figure(...)
#once fig is added, then don't need to do it again. Using the same figure, add layers onto for the rest.
from api_keys import g_key
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure(center=(51.5, -0.1), zoom_level = 2)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
locations

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#View the dataframe
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,deming,32.2687,-107.7586,87.80,8,1,1621122586,18.41,US
1,ushuaia,-54.8000,-68.3000,42.80,70,40,1621122294,8.05,AR
2,port hardy,50.6996,-127.4199,57.20,72,75,1621122586,12.66,CA
3,torbay,47.6666,-52.7314,41.00,100,75,1621122586,3.44,CA
4,bluff,-46.6000,168.3333,48.00,82,37,1621122587,5.01,NZ
...,...,...,...,...,...,...,...,...,...
544,gao,16.6362,1.6370,88.88,27,40,1621123266,2.33,ML
545,abeche,13.8292,20.8324,81.28,7,1,1621123267,8.19,TD
546,ntchisi,-13.5278,33.9149,53.85,78,47,1621123267,1.36,MW
547,korla,41.7597,86.1469,61.77,39,100,1621123267,11.30,CN


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

fit_df = weather_df.loc[(weather_df['Max Temp']>70) & (weather_df['Max Temp']<90) & (weather_df['Wind Speed']<10) \
                       & (weather_df['Cloudiness'] == 0)]
clean_fit_df = fit_df.dropna()
clean_fit_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
108,saint-philippe,-21.3585,55.7679,75.20,68,0,1621122733,9.22,RE
128,saint-pierre,-21.3393,55.4781,73.40,68,0,1621122737,9.22,RE
134,koungou,-12.7336,45.2042,77.00,94,0,1621122739,4.61,YT
201,doha,25.2867,51.5333,89.60,48,0,1621122563,1.14,QA
268,riyadh,24.6877,46.7219,89.60,17,0,1621122870,5.75,SA
306,tall kayf,37.0833,41.0667,71.13,23,0,1621122962,2.44,TR
317,conde,-7.2597,-34.9075,75.20,94,0,1621122965,5.75,BR
345,saint-leu,-21.1500,55.2833,75.20,68,0,1621123032,9.22,RE
442,quelimane,-17.8786,36.8883,73.40,94,0,1621123176,4.61,MZ
488,isa khel,36.6702,68.8443,71.20,37,0,1621123186,1.74,AF


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
#Add a "Hotel Name" column to the DataFrame.
hotel_pre_clean_df = pd.DataFrame(clean_fit_df)
hotel_pre_clean_df['Hotel Name'] = ''

hotel_pre_clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country,Hotel Name
108,saint-philippe,-21.3585,55.7679,75.20,68,0,1621122733,9.22,RE,
128,saint-pierre,-21.3393,55.4781,73.40,68,0,1621122737,9.22,RE,
134,koungou,-12.7336,45.2042,77.00,94,0,1621122739,4.61,YT,
201,doha,25.2867,51.5333,89.60,48,0,1621122563,1.14,QA,
268,riyadh,24.6877,46.7219,89.60,17,0,1621122870,5.75,SA,
306,tall kayf,37.0833,41.0667,71.13,23,0,1621122962,2.44,TR,
317,conde,-7.2597,-34.9075,75.20,94,0,1621122965,5.75,BR,
345,saint-leu,-21.1500,55.2833,75.20,68,0,1621123032,9.22,RE,
442,quelimane,-17.8786,36.8883,73.40,94,0,1621123176,4.61,MZ,
488,isa khel,36.6702,68.8443,71.20,37,0,1621123186,1.74,AF,


In [41]:
# Set base URL for API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_pre_clean_df.iterrows():

    # get locations from df
    location = f"{row['Lat']},{row['Lng']}"
    print(location)
                    
    # set param as location
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']
    
    
    try:

        print(f" closest to {location} is {results[0]['name']}.")
        hotel_pre_clean_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_pre_clean_df.loc[index, 'Hotel Name'] = ''
        

print("------------")

-21.3585,55.7679
Retrieving Results for Index 108: -21.3585,55.7679.
 closest to -21.3585,55.7679 is Chambres d'hôte "La Trinité".
-21.3393,55.4781
Retrieving Results for Index 128: -21.3393,55.4781.
 closest to -21.3393,55.4781 is Lindsey Hôtel.
-12.7336,45.2042
Retrieving Results for Index 134: -12.7336,45.2042.
 closest to -12.7336,45.2042 is Villa Netibor.
25.2867,51.5333
Retrieving Results for Index 201: 25.2867,51.5333.
 closest to 25.2867,51.5333 is Movenpick Hotel Doha.
24.6877,46.7219
Retrieving Results for Index 268: 24.6877,46.7219.
 closest to 24.6877,46.7219 is InterContinental Riyadh, an IHG Hotel.
37.0833,41.0667
Retrieving Results for Index 306: 37.0833,41.0667.
Missing field/result... skipping.
-7.2597,-34.9075
Retrieving Results for Index 317: -7.2597,-34.9075.
 closest to -7.2597,-34.9075 is Pousada Beija Flor.
-21.15,55.2833
Retrieving Results for Index 345: -21.15,55.2833.
 closest to -21.15,55.2833 is La Caz Oceane.
-17.8786,36.8883
Retrieving Results for Index 44

In [54]:
# View dataframe
hotel_pre_clean_df

# Replace empty cells as NaN
hotel_pre_clean_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_pre_clean_df

# Drop rows with no hotels nearby
hotel_df = hotel_pre_clean_df.dropna(subset=['Hotel Name'])
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country,Hotel Name
108,saint-philippe,-21.3585,55.7679,75.2,68,0,1621122733,9.22,RE,"Chambres d'hôte ""La Trinité"""
128,saint-pierre,-21.3393,55.4781,73.4,68,0,1621122737,9.22,RE,Lindsey Hôtel
134,koungou,-12.7336,45.2042,77.0,94,0,1621122739,4.61,YT,Villa Netibor
201,doha,25.2867,51.5333,89.6,48,0,1621122563,1.14,QA,Movenpick Hotel Doha
268,riyadh,24.6877,46.7219,89.6,17,0,1621122870,5.75,SA,"InterContinental Riyadh, an IHG Hotel"
317,conde,-7.2597,-34.9075,75.2,94,0,1621122965,5.75,BR,Pousada Beija Flor
345,saint-leu,-21.1500,55.2833,75.2,68,0,1621123032,9.22,RE,La Caz Oceane
442,quelimane,-17.8786,36.8883,73.4,94,0,1621123176,4.61,MZ,Hotel Flamingo
488,isa khel,36.6702,68.8443,71.2,37,0,1621123186,1.74,AF,راعون انصاف شرکتRAGON INSAF


In [55]:
#look at google gmaps documentation
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))